# Imports & Data

**Imports**

In [ ]:
# External Libraries
import os
import matplotlib.pyplot as plt
import numpy as np
from time import process_time
# My Code
from gmres import GMRES
from msr_reader import msr_reader

**Read Matrices**

In [ ]:
# Paths
directory = os.getcwd() + "\\matrices\\"
matrix_files = ["cg_matrix_msr_1.txt", # 0
            "cg_matrix_msr_2.txt", # 1
            "gmres_matrix_msr_1.txt", # 2
            "msr_test_non_symmetric.txt", # 3
            "msr_test_symmetric.txt"] # 4

# Read each matrix
matrices = []
for file in matrix_files:
    path = directory + f"{file}"
    matrices.append(msr_reader(path))

# GMRES

**Common Settings**

In [ ]:
# Types of preconditioning applied
preconditioners = [None,
                   "jacobi",
                   "gauss_seidel",
                   "ilu0"]

nice_preconds = ["No Preconditioner",
                 "Jacobi",
                 "Gauss-Seidel",
                 "ILU(0)"]

# Further Specifications
A = matrices[2]
n = A.shape[0]
x = np.ones(n)
x0 = np.zeros(n)
b = A.dot(x)
tol = 1e-8

# GMRES

**Task 1: full GMRES**

For the full GMRES method, with and without preconditioning, plot the relative residual
against iteration index on a semi-log scale.

In [ ]:
# Max number of Krylov vectors
ms = [600]

# Timing
durations = []
last_time = process_time()
# Loop
results = []
for m in ms:
    for preconditioner in preconditioners:
        x_approx, errors = GMRES(A, b, x0, m, tol, preconditioner, max_iterations=1000)
        results.append(errors)
        # Status message
        durations.append(round(process_time() - last_time, 3))
        last_time = process_time()
        n_conv = len(errors)
        print(f"Test 1: sek={durations[-1]}, m={m}, prec={preconditioner} iterations={n_conv} ok")

**Results**

<br>Test 1: sek=39.047, m=600, prec=None iterations=562 ok
<br>Test 1: sek=3.094, m=600, prec=jacobi iterations=44 ok
<br>Test 1: sek=18.797, m=600, prec=gauss_seidel iterations=26 ok
<br>Test 1: sek=4.328, m=600, prec=ilu0 iterations=17 ok


In [ ]:
# Print & Plot results
plt.figure(figsize=(8, 4))

for i, errors in enumerate(results):
    # Print number of iterations for convergence
    n_conv = len(errors)
    iterations = range(n_conv)
    plt.plot(iterations, np.abs(errors), label=f'{nice_preconds[i]}')

# Grid and Scale
plt.yscale('log')
plt.grid(True)
plt.xlim([0, 600])
plt.ylim([1e-8, 1])
tick_width = 50
plt.xticks([tick_width*i for i in range(int(650/tick_width))])

# Annotations
plt.xlabel('Iteration')
plt.ylabel('Residual Norm (log scale)')
# plt.title(f'Convergence of full GMRES with and without preconditioning')
plt.legend()

# Save and Plot
filename = "conv_full_gmres.png"
plt.savefig(filename, dpi=300)
plt.show()

**Task 2: GMRES(m)**

For GMRES(m), test and compare the following restart parameters m = 10, 50, 200 with
the runtime required by the full GMRES.

In [ ]:
# Max number of Krylov vectors
ms = [10, 50, 200]

# Timing
durations = []
last_time = process_time()

# Loop
results = []
for m in ms:
    for preconditioner in preconditioners:
        x_approx, errors = GMRES(A, b, x0, m, tol, preconditioner, max_iterations=10000)
        # Status message
        durations.append(round(process_time() - last_time, 3))
        last_time = process_time()
        n_conv = len(errors)
        if abs(errors[-1]) > 1:
            print(f"Test 2: sek={durations[-1]}, m={m}, prec={preconditioner} iterations={n_conv} diverged")
        else:
            results.append(errors)
            print(f"Test 2: sek={durations[-1]}, m={m}, prec={preconditioner} iterations={n_conv} ok")


**Results**

<br>Test 2: sek=259.828, m=10, prec=None iterations=5929 ok
<br>Test 2: sek=0.781, m=10, prec=jacobi iterations=12 diverged
<br>Test 2: sek=17.531, m=10, prec=gauss_seidel iterations=12 diverged
<br>Test 2: sek=4.156, m=10, prec=ilu0 iterations=12 diverged
<br>Test 2: sek=84.688, m=50, prec=None iterations=1813 ok
<br>Test 2: sek=4.188, m=50, prec=jacobi iterations=44 ok
<br>Test 2: sek=19.906, m=50, prec=gauss_seidel iterations=26 ok
<br>Test 2: sek=4.172, m=50, prec=ilu0 iterations=17 ok
<br>Test 2: sek=41.531, m=200, prec=None iterations=835 ok
<br>Test 2: sek=2.734, m=200, prec=jacobi iterations=44 ok
<br>Test 2: sek=18.344, m=200, prec=gauss_seidel iterations=26 ok
<br>Test 2: sek=4.297, m=200, prec=ilu0 iterations=17 ok


**m=10**

In [ ]:
# Print & Plot results
plt.figure(figsize=(8, 4))

# Data
errors = results[0]
n_conv = len(errors)
iterations = range(n_conv)
plt.plot(iterations, np.abs(errors), label=f"No Preconditioner")

# Grid and Scale
plt.yscale('log')
plt.grid(True)
plt.xlim([0, 6000])
plt.ylim([1e-8,1])
tick_width = 500
plt.xticks([tick_width*i for i in range(int(6500/tick_width))])

# Annotations
plt.xlabel('Iteration')
plt.ylabel('Residual Norm (log scale)')
# plt.title(f'Convergence of GMRES(10) with and without preconditioning')
plt.legend()

# Save and Plot
filename = "conv_gmres_10.png"
plt.savefig(filename, dpi=300)
plt.show()

**m=50**

In [ ]:
# Print & Plot results
plt.figure(figsize=(8, 4))

# Data
for i in range(4):
    errors = results[1+i]
    n_conv = len(errors)
    iterations = range(n_conv)
    plt.plot(iterations, np.abs(errors), label=f"{nice_preconds[i]}")

# Grid and Scale
plt.yscale('log')
plt.grid(True)
plt.xlim([0, 100])
plt.ylim([1e-8, 1])
tick_width = 10
plt.xticks([tick_width*i for i in range(int(110/tick_width))])

# Annotations
plt.xlabel('Iteration')
plt.ylabel('Residual Norm (log scale)')
# plt.title(f'Convergence of GMRES(50) with and without preconditioning')
plt.legend()

# Save and Plot
filename = "conv_gmres_50.png"
plt.savefig(filename, dpi=300)
plt.show()

In [ ]:
# Print & Plot results
plt.figure(figsize=(8, 4))
plt.cla()

# Data
for i in range(4):
    errors = results[5+i]
    n_conv = len(errors)
    iterations = range(n_conv)
    plt.plot(iterations, np.abs(errors), label=f"{nice_preconds[i]}")

# Grid and Scale
plt.yscale('log')
plt.grid(True)
plt.xlim([0, 100])
plt.ylim([1e-8, 1])
tick_width = 10
plt.xticks([tick_width*i for i in range(int(110/tick_width))])

# Annotations
plt.xlabel('Iteration')
plt.ylabel('Residual Norm (log scale)')
# plt.title(f'Convergence of GMRES(200) with and without preconditioning')
plt.legend()

# Save and Plot
filename = "conv_gmres_200.png"
plt.savefig(filename, dpi=300)
plt.show()

**Task 3: Orthogonality**

For full GMRES (without preconditioning): check the orthogonality of the Krylov vectors.

**Hints**: Plot the computed values of v1 · vk against k on a semi-log scale.

In [ ]:
# Max number of Krylov vectors
m = 600

# Timing
durations = []
last_time = process_time()
# Loop
results = []
preconditioner = None
x_approx, dot_products = GMRES(A, b, x0, m, tol, preconditioner, max_iterations=1000, orthogonality=True)
# Status message
durations.append(round(process_time() - last_time, 3))
last_time = process_time()
n_conv = len(dot_products)
print(f"Test 1: sek={durations[-1]}, m=600, prec={preconditioner} iterations={n_conv} ok")


In [ ]:
print(f"Final orthogonality error: {dot_products[-1]}")

In [ ]:
# Print & Plot results
plt.figure(figsize=(8, 4))
plt.cla()

# Data
n_conv = len(dot_products)
iterations = range(n_conv)
plt.plot(iterations, np.abs(dot_products))

# Grid and Scale
plt.yscale('log')
plt.grid(True)
plt.xlim([0, 600])
plt.ylim([1e-16, 1e-8])
tick_width = 50
plt.xticks([tick_width*i for i in range(int(650/tick_width))])

# Annotations
plt.xlabel('Iteration')
plt.ylabel(r'Dot product $v_0\cdot v_i$')
# plt.title(r'Orthogonality check for GMRES without preconditioning: $v_0\cdot v_i$')
# plt.legend()

# Save and Plot
filename = "conv_gmres_orthogonality.png"
plt.savefig(filename, dpi=300)
plt.show()

# Conjugate Gradient Method

In [ ]:
import os
from time import process_time
import numpy as np
from msr_reader import msr_reader
import matplotlib.pyplot as plt
from conjugate_gradient import conjugate_gradient


In [ ]:
# Paths
directory = os.getcwd() + "\\matrices\\"
matrix_files = ["cg_matrix_msr_1.txt", # 0
                "cg_matrix_msr_2.txt", # 1
                "gmres_matrix_msr_1.txt", # 2
                "msr_test_non_symmetric.txt", # 3
                "msr_test_symmetric.txt"] # 4

# Read each matrix
matrices = []
for file in matrix_files:
    path = directory + f"{file}"
    matrices.append(msr_reader(path))
print("matrices read")

## **Matrix 1**

In [ ]:
# Create Initial Conditions
A = matrices[0]
n = A.shape[0]
x = np.ones(n)
x0 = np.zeros(n)
b = A.dot(x)
tol = 1e-8
m = 100000 # will stop earlier when tol (= 1e-8) is reached

In [ ]:
# Solve
s = process_time()
x_approx, a_norm_errors, residuals, rel_residuals = conjugate_gradient(A, b, x0, m, tol)
print(f"CG 1: {round(process_time()-s,3)} sec, {len(a_norm_errors)} iterations")
#print(f"Test 1: sek={durations[-1]}, m=600, prec={preconditioner} iterations={n_conv} ok")


In [ ]:
# calculate r_norms
r_norms = []
for r in residuals:
    r_norms.append(np.linalg.norm(r))

## **Matrix 2**

In [ ]:
# Create Initial Conditions
A = matrices[1]
n = A.shape[0]
x = np.ones(n)
x0 = np.zeros(n)
b = A.dot(x)
tol = 1e-8
m = 100000 # will stop earlier when tol (= 1e-8) is reached

In [ ]:
# Solve
s = process_time()
x_approx2, a_norm_errors2, residuals2, rel_residuals2 = conjugate_gradient(A, b, x0, m, tol)
print(f"CG 2: {round(process_time()-s,3)} sec, {len(a_norm_errors2)} iterations")

In [ ]:
# calculate r_norms
r_norms2 = []
for r in residuals2:
    r_norms2.append(np.linalg.norm(r))

## **Plotting**

In [ ]:
# Print & Plot results
plt.figure(figsize=(8, 5))

c_map1 = ["#0083b3", "#12befd", "#9ae1fa"]
c_map2 = ["#9800b3", "#e134ff", "#ee91ff"]

# MATRIX 1
n_conv = len(a_norm_errors)
iterations = range(n_conv)
plt.plot(iterations, np.abs(r_norms), label=r"Residual norm ($M_1$)", color=c_map1[0])
plt.plot(iterations, np.abs(a_norm_errors), label=r"A-norm ($M_1$)", color=c_map1[1]) # M1
plt.plot(iterations, np.abs(rel_residuals), label=r"Relative Residuals ($M_1$)", color=c_map1[2])

# MATRIX 2
n_conv = len(a_norm_errors2)
iterations = range(n_conv)
plt.plot(iterations, np.abs(r_norms2), label=r"Residual norm ($M_2$)", color=c_map2[0])
plt.plot(iterations, np.abs(a_norm_errors2), label=r"A-norm ($M_2$)", color=c_map2[1])
plt.plot(iterations, np.abs(rel_residuals2), label=r"Relative Residuals ($M_2$)", color=c_map2[2])

# Grid and Scale
plt.yscale('log')
plt.ylim([1e-8, 1e10])
plt.grid(True)
plt.xticks(range(0, 11000, 1000))

# Annotations
plt.xlabel('Iteration')
plt.ylabel('Error Norm')
plt.legend()

# Save and Plot
filename = f"cg_convergence.png"
plt.tight_layout()
plt.savefig(filename, dpi=300)
plt.show()